#importando Bibliotecas

In [15]:
# Importar bibliotecas de manipulação de dados
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Importar detectores de outliers
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.linear_model import SGDOneClassSVM

# Importar indicadores de qualidade de clusterização
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score
from sklearn.preprocessing import MinMaxScaler

# Importar indicadores de desempenho de classificação
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Importar bibliotecas adicionais
from time import sleep
import math
import csv


#Funções auxiliares

In [16]:
# Função para criar um arquivo CSV
def criar_csv(nome_arquivo, cabecalho):
    with open(nome_arquivo, 'w', newline='') as arquivo_csv:
        escritor_csv = csv.writer(arquivo_csv)
        escritor_csv.writerow(cabecalho)
# Função para abrir um arquivo CSV existente e adicionar novos dados

def abrir_csv_para_edicao(nome_arquivo):
    with open(nome_arquivo, 'a', newline='') as arquivo_csv:
        escritor_csv = csv.writer(arquivo_csv)
        #OneClassSVM
        array_kernel = ['linear' , 'poly' , 'rbf' ]
        nu_atual = 0.01
        for c in array_kernel:
            while nu_atual <= 0.5:
              y_test = OneClassSVM(gamma='auto', kernel=c, nu=nu_atual).fit_predict(x_train)
              y_test_final = [0 if i == 1 else 1 for i in y_test]

              if(sum(y_test_final) > 0 and 0 in y_test_final):

                #Silhouette Coeficient
                silhouette_value = silhouette_score(x_train, y_test_final, metric='euclidean')

                #calinski Score
                calinski_value = calinski_harabasz_score(x_train, y_test_final)

                #Davies
                davies_value = davies_bouldin_score(x_train, y_test_final)

                linha = 'OneClassSVM,' + "kernel;" +c+ ';nu;' + str(nu_atual) + ',' + str(silhouette_value) + ',' + str(calinski_value) + ',' + str(davies_value)
                dados = linha.split(',')
                escritor_csv.writerow(dados)
                nu_atual =  round(nu_atual + 0.001 ,3)
            nu_atual = 0.01
        #SGDOneClassSVM
        learning_rate = ['constant','optimal', 'invscaling' ,'adaptive']

        nu_atual = 0.01
        for c in learning_rate:
          while (nu_atual <= 0.5):

            y_test = SGDOneClassSVM(learning_rate = c, nu = nu_atual, eta0= 0.1, random_state=0).fit_predict(x_train)
            y_test_final = [0 if i == 1 else 1 for i in y_test]
            if(sum(y_test_final) > 0 and 0 in y_test_final):
              #Silhouette Coeficient
              silhouette_value = silhouette_score(x_train, y_test_final, metric='euclidean')

              #calinski Score
              calinski_value = calinski_harabasz_score(x_train, y_test_final)

              #Davies
              davies_value = davies_bouldin_score(x_train, y_test_final)

              linha = 'SGDOneClassSVM,' + "Learning_rate;" +c+ ';nu;' + str(nu_atual) + ',' + str(silhouette_value) + ',' + str(calinski_value) + ',' + str(davies_value)
              dados = linha.split(',')
              escritor_csv.writerow(dados)
            nu_atual =  round(nu_atual + 0.001 ,3)
          nu_atual = 0.01


        array_algorithm = ['auto', 'ball_tree', 'kd_tree','brute']

        for c in array_algorithm:
          b = 0
          while b < 10:
            b = b + 1
            y_test = LocalOutlierFactor(n_neighbors=b, algorithm = c).fit_predict(x_train)
            y_test_final = [0 if i == 1 else 1 for i in y_test]

            if(sum(y_test_final) > 0 and 0 in y_test_final):
              #Silhouette Coeficient
              silhouette_value = silhouette_score(x_train, y_test_final, metric='euclidean')

              #calinski Score
              calinski_value = calinski_harabasz_score(x_train, y_test_final)

              #Davies
              davies_value = davies_bouldin_score(x_train, y_test_final)
              linha = 'LocalOutlierFactor,' + "array_algorithm;" + c + ";"+ "n_neighbors;" +str(b)+ ',' + str(silhouette_value) + ',' + str(calinski_value) + ',' + str(davies_value)
              dados = linha.split(',')
              escritor_csv.writerow(dados)




def manipulacao( nomeArq ):
  csv_dados = nomeArq
  csv = pd.read_csv(csv_dados, sep=",")

  Colunas = ['Silhouette','Calinski','Davies']
  data = csv[Colunas]
  scaler = MinMaxScaler()
  scaler.fit(data)
  x = scaler.transform(data)
  Silhouette = x[:,0:1]
  Calinski =  x[:,1:2]
  Davies =  x[:,2:3]



  Silhouette = Silhouette.reshape(Silhouette.shape[1],Silhouette.shape[0])[0]
  Calinski = Calinski.reshape(Calinski.shape[1],Calinski.shape[0])[0]
  Davies = Davies.reshape(Davies.shape[1],Davies.shape[0])[0]

  Davies = 1-Davies

  Media_indicadores = (Calinski + Silhouette)/2

  df = pd.read_csv(nomeArq)

  # Criar os dados para as novas colunas
  novas_colunas = {
      'Silhouette_normalizado': Silhouette,
      'Calinski_normalizado': Calinski,
      'Davies_normalizado': Davies,
      'Media_indicadores': Media_indicadores
  }



  # Converter o dicionário para um DataFrame
  df_novas_colunas = pd.DataFrame(novas_colunas)

  # Concatenar os DataFrames ao longo das colunas (axis=1)
  df_concatenado = pd.concat([df, df_novas_colunas], axis=1)


  Colunas = ['Modelo','Parametros','Silhouette_normalizado','Davies_normalizado','Calinski_normalizado','Media_indicadores']
  Data_Select_db = df_concatenado[Colunas]
  Data_Select_db_ordenado = Data_Select_db.sort_values(by='Media_indicadores',ascending = False)
  x = Data_Select_db_ordenado.iloc[0][1].split(';')



  if Data_Select_db_ordenado.iloc[0][0] == "OneClassSVM":
    clf = OneClassSVM(kernel = x[1], nu= float(x[3])).fit(x_train)
    y_test = clf.predict(x_test)
    y_atual = [0 if i == 1 else 1 for i in y_test]
    print(confusion_matrix(y_real, y_atual))
    print(classification_report(y_real, y_atual, digits = 4))
  if Data_Select_db_ordenado.iloc[0][0] == "SGDOneClassSVM":
    clf = SGDOneClassSVM(learning_rate = x[1], nu= float(x[3]), eta0= 0.1 ).fit(x_train)
    y_test = clf.predict(x_test)
    y_atual = [0 if i == 1 else 1 for i in y_test]
    print(confusion_matrix(y_real, y_atual))
    print(classification_report(y_real, y_atual, digits = 4))
  if Data_Select_db_ordenado.iloc[0][0] == "LocalOutlierFactor":
    clf = LocalOutlierFactor(array_algorithm = x[1], n_neighbors= int(x[3])).fit(x_train)
    y_test = clf.predict(x_test)
    y_atual = [0 if i == 1 else 1 for i in y_test]
    print(confusion_matrix(y_real, y_atual))
    print(classification_report(y_real, y_atual, digits = 4))
  if Data_Select_db_ordenado.iloc[0][0] == "EllipticEnvelope":
    clf = EllipticEnvelope(contamination = float(x[1])).fit(x_train)
    y_test = clf.predict(x_test)
    y_atual = [0 if i == 1 else 1 for i in y_test]
    print(confusion_matrix(y_real, y_atual))
    print(classification_report(y_real, y_atual, digits = 4))



#Experimento 1

In [17]:
#cap_51
early_warning_url = "cap_51_outlier_10.csv"
early_warning = pd.read_csv(early_warning_url, sep=";")
file = "10"
colunas = [file+'_kurt_total_ips_origem',
           file+'_skw_total_ips_origem',
           file+'_coefficient_variation_total_ips_origem',
           file+'_kurt_total_ips_destino',
           file+'_skw_total_ips_destino',
           file+'_coefficient_variation_total_ips_destino',
           file+'_kurt_total_pacotes',
           file+'_skw_total_pacotes',
           file+'_coefficient_variation_total_pacotes',
              ]
df = early_warning[colunas]
label = early_warning['has_bot']

init = 880
train = 2934
end = 5632

x_train = df[init:train]
x_test = df[train:end]
y_real = label[train:end]


##Executando extração indicadores:

In [18]:
# Nome do arquivo CSV e cabeçalho
nome_arquivo = 'dados_ctu13.csv'
cabecalho = ['Modelo', 'Parametros', 'Silhouette', 'Calinski', 'Davies' ]

# Criar o arquivo CSV com o cabeçalho
criar_csv(nome_arquivo, cabecalho)

# Abrir o arquivo CSV para edição
abrir_csv_para_edicao(nome_arquivo)

print("Arquivo CSV criado e editado com sucesso!")

manipulacao(nome_arquivo)

Arquivo CSV criado e editado com sucesso!


<ipython-input-16-04aa7acf8279>:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = Data_Select_db_ordenado.iloc[0][1].split(';')
<ipython-input-16-04aa7acf8279>:135: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if Data_Select_db_ordenado.iloc[0][0] == "OneClassSVM":


[[2415   67]
 [ 204   12]]
              precision    recall  f1-score   support

           0     0.9221    0.9730    0.9469      2482
           1     0.1519    0.0556    0.0814       216

    accuracy                         0.8996      2698
   macro avg     0.5370    0.5143    0.5141      2698
weighted avg     0.8604    0.8996    0.8776      2698



<ipython-input-16-04aa7acf8279>:141: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if Data_Select_db_ordenado.iloc[0][0] == "SGDOneClassSVM":
<ipython-input-16-04aa7acf8279>:147: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if Data_Select_db_ordenado.iloc[0][0] == "LocalOutlierFactor":
<ipython-input-16-04aa7acf8279>:153: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if Data_Select_db_ordenado.iloc[0][0] == "EllipticEnvelope":


#Experimento 2

In [20]:
#cic
early_warning_url = "cicddos_outlier_10.csv"
early_warning = pd.read_csv(early_warning_url, sep=";")
file = "10"
colunas = [file+'_kurt_total_ips_origem',
           file+'_skw_total_ips_origem',
           file+'_coefficient_variation_total_ips_origem',
           file+'_kurt_total_ips_destino',
           file+'_skw_total_ips_destino',
           file+'_coefficient_variation_total_ips_destino',
           file+'_kurt_total_pacotes',
           file+'_skw_total_pacotes',
           file+'_coefficient_variation_total_pacotes',
              ]
df = early_warning[colunas]
label = early_warning['has_bot']


init = 202
train = 674
end = 1484

x_train = df[init:train]
x_test = df[train:end]
y_real = label[train:end]

##Executando extração indicadores:

In [21]:
# Nome do arquivo CSV e cabeçalho
nome_arquivo = 'dados_cic.csv'
cabecalho = ['Modelo', 'Parametros', 'Silhouette', 'Calinski', 'Davies' ]

# Criar o arquivo CSV com o cabeçalho
criar_csv(nome_arquivo, cabecalho)

# Abrir o arquivo CSV para edição
abrir_csv_para_edicao(nome_arquivo)

print("Arquivo CSV criado e editado com sucesso!")

manipulacao(nome_arquivo)

Arquivo CSV criado e editado com sucesso!
[[531  29]
 [239  11]]
              precision    recall  f1-score   support

           0     0.6896    0.9482    0.7985       560
           1     0.2750    0.0440    0.0759       250

    accuracy                         0.6691       810
   macro avg     0.4823    0.4961    0.4372       810
weighted avg     0.5616    0.6691    0.5755       810



<ipython-input-16-04aa7acf8279>:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = Data_Select_db_ordenado.iloc[0][1].split(';')
<ipython-input-16-04aa7acf8279>:135: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if Data_Select_db_ordenado.iloc[0][0] == "OneClassSVM":
<ipython-input-16-04aa7acf8279>:141: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if Data_Select_db_ordenado.iloc[0][0] == "SGDOneClassSVM":
<ipython-input-16-04aa

In [22]:
# Configurar e treinar o modelo SGD One-Class SVM
clf = SGDOneClassSVM(learning_rate='adaptive', eta0=0.1, nu=0.474 ).fit(x_train)

# Fazer previsões no conjunto de teste
y_test = clf.predict(x_test)

# Converter as previsões para o formato esperado (0 para normal, 1 para outlier)
y_atual = [0 if i == 1 else 1 for i in y_test]

# Imprimir a matriz de confusão e o relatório de classificação
print(confusion_matrix(y_real, y_atual))
print(classification_report(y_real, y_atual, digits=4))


[[532  28]
 [239  11]]
              precision    recall  f1-score   support

           0     0.6900    0.9500    0.7994       560
           1     0.2821    0.0440    0.0761       250

    accuracy                         0.6704       810
   macro avg     0.4860    0.4970    0.4378       810
weighted avg     0.5641    0.6704    0.5762       810

